# Notebook for Peak Reduction with Mixed Devices

## Imports

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pyflexad.models.bess.tesla as tesla_bess
import pyflexad.models.ev.nissan as nissan_ev
import pyflexad.models.ev.tesla as tesla_ev

import pyflexad.models.tclc.generic as generic_tclc
import pyflexad.models.tclh.generic as generic_tclh

from pyflexad.physical.electric_vehicle import ElectricVehicle
from pyflexad.physical.stationary_battery import StationaryBattery
from pyflexad.physical.stationary_battery import BESSUsage

from pyflexad.physical.therm_load_cooling import ThermostaticLoadCooling
from pyflexad.physical.therm_load_cooling import TCLCUsage

from pyflexad.physical.therm_load_heating import ThermostaticLoadHeating
from pyflexad.physical.therm_load_heating import TCLHUsage

from pyflexad.virtual.aggregator import Aggregator
from pyflexad.math.signal_vectors import SignalVectors
from pyflexad.optimization.vertex_based_power_controller import VertexBasedPowerController
from pyflexad.optimization.centralized_power_controller import CentralizedPowerController
from pyflexad.system.household_data import HouseholdsData
from pyflexad.system.ev_data import EVData
from pyflexad.utils.file_utils import FileUtils
from pyflexad.utils.algorithms import Algorithms

## Scenario Description
We consider a scenario with $k$ nodes, each with $h_k$ households. These households may have a stationary battery, an electric vehicle, an air conditioner and a water heater. We assume that the electric vehicles start with a SOC of 60%, while the SOC of the stationary battery is randomly generated. At the end, the BESS must have at least the initial SOC and the EV at least the initial SOC - consumption during trip + charging, where the consumption during the trip and charging are given by the data. 
The air conditioner starts with a set temperature randomly sampled from the interval [19, 21] (°C), where the ambient temperature is 30°C and the dead band is randomly selected from the interval [1.5, 2.5] (°C). The water heater starts with a setpoint temperature randomly sampled from the intervall [49, 51] (°C), where the dead band is randomly selected from the interval [3, 7] (°C), and the demand is randomly selected from the interval [0, 4] (kW) for each time period. The devices, amount of households and nodes can be selected individually in the configurations section. The flexibility within the $k$ nodes are aggregated locally. In a second step, the local flexibilities are aggregated by another aggregator to obtain the overall aggregate flexibility of the system, i.e. we consider a stacked system where the stacked aggregation is applied. The overall objective is to study the effect of the aggregated flexibility to reduce the peak demand of the aggregated load.
For this purpose, the results of the proposed algorithm are compared with those of the centralised approach, i.e. where no aggregation is applied.

## Configurations 

In [ ]:
d = 96
dt = 0.25
algorithm = Algorithms.IABVG_JIT
path_da = os.path.join(FileUtils.data_dir, "processed_da")
path_hh = os.path.join(FileUtils.data_dir, "processed_hh")
path_ev = os.path.join(FileUtils.data_dir, "data_EV")
dates_ev = pd.date_range('12/14/2014, 00:00', periods=d, freq='15 min')
dates_h = pd.date_range('12/14/2009, 00:00', periods=d, freq='15 min')

                                                #sub 1 sub 2 sub 3
info_dict = {
           "Nissan Leaf 6.6kW AC":              [4,   4,   4],
           "Tesla Model Y 11kW AC":             [4,   4,   4],
           "Tesla Model Y 100kW DC":            [0,   0,   0],
           "Tesla Model 3 RWD 11kW AC":         [0,   0,   0],
           "Tesla Model 3 RWD 100kW DC":        [0,   0,   0],
           "Tesla Model S P100D 16.5kW AC":     [4,   4,   4],
           "Tesla Powerwall 2":                 [4,   4,   4],
           "Tesla Powerwall 3":                 [4,   4,   4],
           "Tesla Powerwall+":                  [4,   4,   4],
           "generic tclc":                      [6,   6,   6],
           "generic tclh":                      [6,   6,   6],
           "participants":                      [100, 100, 100]
}
ev_models = nissan_ev.models + tesla_ev.models
bess_models = tesla_bess.models
tclc_models = generic_tclc.models
tclh_models = generic_tclh.models

for key, value in info_dict.items():
    info_dict[key] = np.array(value)

In [ ]:
"""print overall model"""
display(pd.DataFrame(index=[1, 2, 3], data=info_dict))

## Data

In [ ]:
#np.random.seed(3)
signal_vectors = SignalVectors.new(d, g=SignalVectors.g_of_d_exp_2(d))
hh_data = HouseholdsData.from_file(path_hh=path_hh,
                                   n_entities=info_dict["participants"].sum(),
                                   n_time_periods=d)

## Define Flexibilities

In [ ]:
co_esr_list = []
sub_agg_list = []
ev_agg_av = np.zeros(d)
ev_agg_dem = np.zeros(d)
for k in range(len(info_dict["participants"])):
    esr_list = []
    for hardware in ev_models:
        if hardware.name in info_dict:
            ev_data = EVData.from_file(path_ev=path_ev, n_entities=info_dict[hardware.name][k], n_time_periods=d, dt=dt, dates=dates_ev)
    
            ev_agg_av += ev_data.availability.sum(axis=0)
            ev_agg_dem += ev_data.power_demand.sum(axis=0)
            for i in range(info_dict[hardware.name][k]):
                esr = ElectricVehicle.with_charging(hardware=hardware,
                                                    initial_capacity=0.5 * hardware.max_capacity,
                                                    availability=ev_data.availability[i, :],
                                                    power_demand=ev_data.power_demand[i, :],
                                                    charging=ev_data.charging[i, :], d=d, dt=dt)
                esr_list.append(esr)
    
    for hardware in bess_models:
        if hardware.name in info_dict:
            for i in range(info_dict[hardware.name][k]):
                init_capacity = np.random.uniform(0.5 * hardware.max_capacity, hardware.max_capacity)
                usage = BESSUsage(initial_capacity=init_capacity, final_capacity=init_capacity, d=d, dt=dt)
                esr = StationaryBattery.new(hardware=hardware, usage=usage)
                esr_list.append(esr)
    
    for hardware in tclc_models:
        if hardware.name in info_dict:
            for i in range(info_dict[hardware.name][k]):
                usage = TCLCUsage.from_celsius(theta_r_deg_c=20, theta_a_deg_c=30,
                                               theta_0_deg_c=np.random.uniform(19, 21),
                                               delta=np.random.uniform(1.5, 2.5), d=d, dt=dt)
                esr = ThermostaticLoadCooling.new(hardware=hardware, usage=usage)
                esr_list.append(esr)
    
    for hardware in tclh_models:
        if hardware.name in info_dict:
            for i in range(info_dict[hardware.name][k]):    
                demand = np.random.uniform(0, 3, d)
                usage = TCLHUsage.from_celsius(theta_r_deg_c=50, theta_a_deg_c=30,
                                               theta_0_deg_c=np.random.uniform(49, 51),
                                               delta=np.random.uniform(3, 7), demand=demand, d=d, dt=dt)
                esr = ThermostaticLoadHeating.new(hardware=hardware, usage=usage)
                esr_list.append(esr)
        
    """add items to centralized optimization list"""
    co_esr_list += esr_list

    """add items to decentralized optimization list"""
    sub_agg_list.append(Aggregator.from_physical(esr_list, signal_vectors=signal_vectors, algorithm=algorithm))


## Compute Results

In [ ]:
top_agg = Aggregator.aggregate(sub_agg_list, algorithm=algorithm)

vo = VertexBasedPowerController(power_demand=hh_data.power_demand)
co = CentralizedPowerController(power_demand=hh_data.power_demand)

vo_power = vo.optimize(top_agg)
co_power = co.optimize(co_esr_list)

## Plot Results

In [ ]:
plt.figure()
df = pd.DataFrame(index=dates_h, 
                  data=hh_data.power_demand.sum(axis=0), 
                  columns=["aggregated load"])

"""we assume that the additional energy needed is considered in the demand curve"""
df_agg = pd.DataFrame(
    index=dates_h,
    data=hh_data.power_demand.sum(axis=0) + vo_power - vo_power.mean() * np.ones(d),
    columns=["aggregation"])

df_exact = pd.DataFrame(
    index=dates_h,
    data=hh_data.power_demand.sum(axis=0) + co_power - co_power.mean() * np.ones(d),
    columns=["centralized"])

df = pd.concat([df, df_agg], axis=1)
df = pd.concat([df, df_exact], axis=1)

"""only interested in demand, negative values are supply not demand"""
df["aggregation"] = df["aggregation"].apply(lambda x: 0 if x < 0 else x) 
df["centralized"] = df["centralized"].apply(lambda x: 0 if x < 0 else x)

df.plot.area(stacked=False, ylabel="demand (kW)", grid=True)
plt.show()

In [ ]:
print(f"exact peak: {np.round(np.linalg.norm(df['centralized'].values, np.inf),2)} kW")
print(f"approx peak: {np.round(np.linalg.norm(df['aggregation'].values, np.inf),2)} kW")

In [ ]:
plt.figure()
df = pd.DataFrame(index=dates_h, data=ev_agg_av, columns=["EV availability"])
df.plot(style="-o",grid=True)
plt.show()

In [ ]:
plt.figure()
df = pd.DataFrame(index=dates_h, data=ev_agg_dem, columns=["EV trip consumption"])
df.plot.area(ylabel="power (kW)",grid=True, color="orange")
plt.show()